In [ ]:
!pip3 install --upgrade  pydantic google-cloud-aiplatform google-cloud-storage "shapely<2" tensorflow-text==2.9.0

from google.colab import auth
auth.authenticate_user()

In [ ]:
exported_files = dataset.export_data(output_dir=BUCKET_URI)
print("Following files were exported")
print(exported_files)

In [ ]:
from google.cloud import storage 

def read_gcs_with_full_path(storage_client, filepath):

  bucket, blob_name = filepath.split("/")[2], "/".join(filepath.split("/")[3:])
  bucket = storage_client.bucket(bucket)
  blob = bucket.blob(blob_name)

  with blob.open("r") as f:
      data = f.read()

  return data

def read_gcs_from_exported_dataset(exported_files: List[str]):

  storage_client = storage.Client()

  for filepath in exported_files:

    print("Reading the JSON line file")
    data = read_gcs_with_full_path(storage_client, filepath)

    print("Building instances")
    instances = [ TextClassificationSample.parse_raw(ins) for ins in data.split("\n")]

    print("Building data subsets")
    datasets = {"training":{"inputs":[], "labels": []}, "validation":{"inputs":[], "labels": []}, "test":{"inputs":[], "labels": []}}
    label_map = {"positive": 1, "negative": 0}
    for ins in instances:
      txt = read_gcs_with_full_path(storage_client, ins.textGcsUri)
      datasets[ins.dataItemResourceLabels.ml_use]["inputs"].append(txt)
      datasets[ins.dataItemResourceLabels.ml_use]["labels"].append(label_map[ins.classificationAnnotation.displayName])

  return datasets

datasets = read_gcs_from_exported_dataset(exported_files)

for k, v in datasets.items():
  print(f"{k} dataset contains {len(v['inputs'])} samples")

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf 
# Unless this import is here, the following error comes up
# Error Op type not registered 'CaseFoldUTF8' in binary running on 932fd13e3432. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
# You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.
import tensorflow_text

tf.keras.backend.clear_session()

def download_model() -> tf.keras.Model:
  preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
  encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/2",
    trainable=False)

  return preprocessor, encoder

preprocessor, encoder = download_model()

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)

encoder_inputs = preprocessor(text_input)

encoder_outputs = encoder(encoder_inputs)

pooled_output = encoder_outputs["pooled_output"]      # [batch_size, 512].
#sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 512].
hidden_layer = tf.keras.layers.Dense(256, activation="gelu")
classif_layer = tf.keras.layers.Dense(1, activation="sigmoid")
hidden_out = hidden_layer(pooled_output)
final_out = classif_layer(hidden_out)

model = tf.keras.Model(inputs=text_input, outputs=final_out)
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics="accuracy")

print(model.summary())